In [0]:
import numpy as np
import os
import shutil
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, models, transforms

import kaggle

In [0]:
# skip these lines if not using Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# skip this line if not using Colab
!cp 'drive/My Drive/Colab Notebooks/Kaggle API/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 34.3MB/s]
 94% 254M/271M [00:01<00:00, 187MB/s]
100% 271M/271M [00:01<00:00, 143MB/s]
 99% 538M/543M [00:07<00:00, 64.2MB/s]
100% 543M/543M [00:07<00:00, 72.1MB/s]


In [0]:
!mkdir dogs-cats
!mv test1.zip dogs-cats
!mv train.zip dogs-cats
!mv sampleSubmission.csv dogs-cats

In [0]:
cd dogs-cats

/content/dogs-cats


In [0]:
# extract all images
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as zipObj:
  zipObj.extractall()
with ZipFile('test1.zip', 'r') as zipObj:
  zipObj.extractall()

In [0]:
train_dir = 'train/'
test_dir = 'test1/'

In [0]:
# create cats and dogs folders in train folder
if not os.path.exists(train_dir + 'cats'):
  os.mkdir(train_dir + 'cats')
if not os.path.exists(train_dir + 'dogs'):
  os.mkdir(train_dir + 'dogs')
  
# move images to respective dogs and cats folder, in order to use ImageFolder from torch
for filename in os.listdir(train_dir):
  if filename.startswith('cat.'):
    shutil.move(train_dir + filename, train_dir + 'cats')
  if filename.startswith('dog.'):
    shutil.move(train_dir + filename, train_dir + 'dogs')

In [0]:
# get all training data to calculate mean and std
transform = transforms.Compose([
  transforms.Resize([224, 224]),
  transforms.ToTensor(),
  ])
images = datasets.ImageFolder(train_dir, transform)
dataloader = DataLoader(images, batch_size=64)

In [0]:
# calculate mean and std
def calc_mean_std(dataloader):
  mean, std = 0, 0
  num_samples = 0
  for X, _ in dataloader:
    X = X.view(X.size(0), X.size(1), -1)
    mean += torch.mean(X, 2).sum(0)
    std += torch.std(X, 2).sum(0)
    num_samples += X.size(0)
  return mean/num_samples, std/num_samples

In [0]:
mean, std = calc_mean_std(dataloader)
print('Mean: ', mean, ' Std: ', std)

Mean:  tensor([0.4883, 0.4551, 0.4170])  Std:  tensor([0.2257, 0.2211, 0.2214])


In [0]:
# take 23000 images for training, 2000 for validation
NUM_TRAIN = 23000
NUM_TOTAL = 25000

# data preprocessing and augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2257, 0.2211, 0.2214])
    ]),
    'test': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2257, 0.2211, 0.2214])                            
    ])
}

train_imgs = datasets.ImageFolder(train_dir, data_transforms['train'])
indices = list(range(NUM_TOTAL))
np.random.shuffle(indices)
train_indices, val_indices = indices[:NUM_TRAIN], indices[NUM_TRAIN:]

train_loader = DataLoader(train_imgs, batch_size=64, sampler=SubsetRandomSampler(train_indices))
val_loader = DataLoader(train_imgs, batch_size=64, sampler=SubsetRandomSampler(val_indices))

In [0]:
def validate(model, loader):
  model = model.to(device)
  model.eval()
  num_corrects = 0
  num_samples = 0
  with torch.no_grad():
    for X, y in loader:
      X = X.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      output = model(X)
      preds = torch.max(output, 1)[1]
      num_corrects += (preds == y).sum().item()
      num_samples += X.size(0)
    model.train()
    return 100. * float(num_corrects) / float(num_samples)

def train(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, best_model_name, num_epochs=1, print_every=1):
  model = model.to(device)
  model.train()
  best_acc = 0
  best_state_dict = model.state_dict()
  since = time.time()
  for epoch in range(num_epochs):
    print('Epoch %d' %epoch)
    print('-—-—-—')
    num_corrects = 0
    num_samples = 0
    running_loss = 0
    lr_scheduler.step()
    since_epoch = time.time()
    for t, (X, y) in enumerate(train_loader):
      X = X.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      optimizer.zero_grad()
      output = model(X)
      preds = torch.max(output, 1)[1]
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      if t % print_every == 0:
        acc_val = validate(model, val_loader)
        print('Iter %d, train loss = %.3f, val acc = %.3f' %(t, running_loss/(t+1), acc_val))
        if acc_val > best_acc:
          best_acc = acc_val
          best_state_dict = model.state_dict()
    acc_val = validate(model, val_loader)
    if acc_val > best_acc:
      best_acc = acc_val
      best_state_dict = model.state_dict()
    end_epoch = time.time()
    print('End of epoch %d, time taken = %.3fs' %(epoch, end_epoch - since_epoch))
    print()
  model.load_state_dict(best_state_dict)
  torch.save(best_state_dict, best_model_name + '-dogs-cats.pth')
  end = time.time()
  print('Training completed, total time taken = %.3fs' %(end - since))
  return model

In [0]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [0]:
# using ResNet18
model_rn18 = models.resnet18(pretrained=True)
for params in model_rn18.parameters():
  params.requires_grad = False

model_rn18.fc = nn.Linear(model_rn18.fc.in_features, 2)
criterion1 = nn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(model_rn18.parameters(), lr=0.01)
lr_scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=1, gamma=0.3)

In [0]:
resnet18 = train(model_rn18, train_loader, val_loader, criterion1, optimizer1, lr_scheduler1, 'resnet18', num_epochs=5, print_every=100)

Epoch 0
-—-—-—
Iter 0, train loss = 0.632, val acc = 50.600
Iter 100, train loss = 0.343, val acc = 95.050
Iter 200, train loss = 0.267, val acc = 96.150
Iter 300, train loss = 0.227, val acc = 96.300
End of epoch 0, time taken = 250.895s

Epoch 1
-—-—-—
Iter 0, train loss = 0.096, val acc = 96.950
Iter 100, train loss = 0.135, val acc = 96.800
Iter 200, train loss = 0.133, val acc = 97.050
Iter 300, train loss = 0.129, val acc = 97.200
End of epoch 1, time taken = 250.914s

Epoch 2
-—-—-—
Iter 0, train loss = 0.128, val acc = 96.950
Iter 100, train loss = 0.119, val acc = 96.900
Iter 200, train loss = 0.122, val acc = 97.300
Iter 300, train loss = 0.122, val acc = 96.750
End of epoch 2, time taken = 251.020s

Epoch 3
-—-—-—
Iter 0, train loss = 0.162, val acc = 97.000
Iter 100, train loss = 0.124, val acc = 96.750
Iter 200, train loss = 0.123, val acc = 96.900
Iter 300, train loss = 0.120, val acc = 96.850
End of epoch 3, time taken = 250.515s

Epoch 4
-—-—-—
Iter 0, train loss = 0.12

In [0]:
# using ResNet34
model_rn34 = models.resnet34(pretrained=True)
for params in model_rn34.parameters():
  params.requires_grad = False

model_rn34.fc = nn.Linear(model_rn34.fc.in_features, 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_rn34.parameters(), lr=0.005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.2)

In [0]:
resnet34 = train(model_rn34, train_loader, val_loader, criterion, optimizer, lr_scheduler, 'resnet34', num_epochs=5, print_every=100)

Epoch 0
-—-—-—
Iter 0, train loss = 0.860, val acc = 53.000
Iter 100, train loss = 0.221, val acc = 97.200
Iter 200, train loss = 0.163, val acc = 97.550
Iter 300, train loss = 0.136, val acc = 97.850
End of epoch 0, time taken = 283.752s

Epoch 1
-—-—-—
Iter 0, train loss = 0.067, val acc = 97.750
Iter 100, train loss = 0.070, val acc = 97.800
Iter 200, train loss = 0.069, val acc = 98.250
Iter 300, train loss = 0.070, val acc = 97.950
End of epoch 1, time taken = 283.890s

Epoch 2
-—-—-—
Iter 0, train loss = 0.068, val acc = 98.350
Iter 100, train loss = 0.072, val acc = 97.900
Iter 200, train loss = 0.071, val acc = 98.000
Iter 300, train loss = 0.069, val acc = 98.150
End of epoch 2, time taken = 284.162s

Epoch 3
-—-—-—
Iter 0, train loss = 0.110, val acc = 97.650
Iter 100, train loss = 0.063, val acc = 98.150
Iter 200, train loss = 0.064, val acc = 98.150
Iter 300, train loss = 0.065, val acc = 98.000
End of epoch 3, time taken = 283.690s

Epoch 4
-—-—-—
Iter 0, train loss = 0.07

In [0]:
# using ResNet50
model_rn50 = models.resnet50(pretrained=True)
for params in model_rn50.parameters():
  params.requires_grad = False

model_rn50.fc = nn.Linear(model_rn50.fc.in_features, 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_rn50.parameters(), lr=0.002)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 74307816.49it/s]


In [0]:
resnet50 = train(model_rn50, train_loader, val_loader, criterion, optimizer, lr_scheduler, 'resnet50', num_epochs=5, print_every=100)

Epoch 0
-—-—-—
Iter 0, train loss = 0.579, val acc = 48.950
Iter 100, train loss = 0.170, val acc = 97.800
Iter 200, train loss = 0.126, val acc = 98.300
Iter 300, train loss = 0.111, val acc = 97.250
End of epoch 0, time taken = 404.666s

Epoch 1
-—-—-—
Iter 0, train loss = 0.058, val acc = 98.300
Iter 100, train loss = 0.057, val acc = 98.500
Iter 200, train loss = 0.062, val acc = 98.400
Iter 300, train loss = 0.062, val acc = 98.650
End of epoch 1, time taken = 403.916s

Epoch 2
-—-—-—
Iter 0, train loss = 0.034, val acc = 98.650
Iter 100, train loss = 0.051, val acc = 98.450
Iter 200, train loss = 0.052, val acc = 98.500
Iter 300, train loss = 0.055, val acc = 98.800
End of epoch 2, time taken = 404.983s

Epoch 3
-—-—-—
Iter 0, train loss = 0.079, val acc = 98.550
Iter 100, train loss = 0.061, val acc = 98.550
Iter 200, train loss = 0.059, val acc = 98.500
Iter 300, train loss = 0.056, val acc = 98.300
End of epoch 3, time taken = 404.780s

Epoch 4
-—-—-—
Iter 0, train loss = 0.13

In [0]:
# copy model state dict
!cp resnet50-dogs-cats.pth '../drive/My Drive/Colab Notebooks/small projects/dogs-cats'

In [0]:
# using VGG16 (with BatchNorm)
model_vgg16_bn = models.vgg16_bn(pretrained=True)
for params in model_vgg16_bn.parameters():
  params.requires_grad = False
  
model_vgg16_bn.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=2, bias=True)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_vgg16_bn.parameters(), lr=0.002)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [0]:
vgg16_bn = train(model_vgg16_bn, train_loader, val_loader, criterion, optimizer, lr_scheduler, 'vgg16_bn', num_epochs=5, print_every=100)

Epoch 0
-—-—-—
Iter 0, train loss = 0.688, val acc = 48.950
Iter 100, train loss = 0.552, val acc = 97.600
Iter 200, train loss = 0.311, val acc = 98.750
Iter 300, train loss = 0.230, val acc = 97.350
End of epoch 0, time taken = 608.749s

Epoch 1
-—-—-—
Iter 0, train loss = 0.067, val acc = 98.100
Iter 100, train loss = 0.048, val acc = 98.850
Iter 200, train loss = 0.045, val acc = 98.800
Iter 300, train loss = 0.043, val acc = 98.950
End of epoch 1, time taken = 603.276s

Epoch 2
-—-—-—
Iter 0, train loss = 0.008, val acc = 98.900
Iter 100, train loss = 0.038, val acc = 99.000
Iter 200, train loss = 0.037, val acc = 99.050
Iter 300, train loss = 0.035, val acc = 99.150
End of epoch 2, time taken = 605.707s

Epoch 3
-—-—-—
Iter 0, train loss = 0.017, val acc = 99.150
Iter 100, train loss = 0.028, val acc = 99.300
Iter 200, train loss = 0.030, val acc = 98.700
Iter 300, train loss = 0.027, val acc = 99.150
End of epoch 3, time taken = 619.488s

Epoch 4
-—-—-—
Iter 0, train loss = 0.01

In [0]:
# copy model state dict
!cp vgg16_bn-dogs-cats.pth '../drive/My Drive/Colab Notebooks/small projects/dogs-cats'

In [0]:
# using DenseNet121
model_dense121 = models.densenet121(pretrained=True)
for params in model_dense121.parameters():
  params.requires_grad = False
  
model_dense121.classifier = nn.Sequential(
    nn.Linear(in_features=1024, out_features=1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=1000, out_features=2, bias=True)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_dense121.parameters(), lr=0.002)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [0]:
dense121 = train(model_dense121, train_loader, val_loader, criterion, optimizer, lr_scheduler, 'dense121', num_epochs=5, print_every=100)

Epoch 0
-—-—-—
Iter 0, train loss = 0.783, val acc = 51.600
Iter 100, train loss = 0.159, val acc = 98.300
Iter 200, train loss = 0.120, val acc = 97.750
Iter 300, train loss = 0.103, val acc = 98.700
End of epoch 0, time taken = 417.909s

Epoch 1
-—-—-—
Iter 0, train loss = 0.176, val acc = 97.750
Iter 100, train loss = 0.066, val acc = 99.150
Iter 200, train loss = 0.060, val acc = 98.750
Iter 300, train loss = 0.059, val acc = 98.700
End of epoch 1, time taken = 418.500s

Epoch 2
-—-—-—
Iter 0, train loss = 0.103, val acc = 98.250
Iter 100, train loss = 0.052, val acc = 98.450
Iter 200, train loss = 0.054, val acc = 99.150
Iter 300, train loss = 0.054, val acc = 99.150
End of epoch 2, time taken = 418.227s

Epoch 3
-—-—-—
Iter 0, train loss = 0.021, val acc = 98.800
Iter 100, train loss = 0.046, val acc = 98.850
Iter 200, train loss = 0.051, val acc = 98.600
Iter 300, train loss = 0.050, val acc = 98.600
End of epoch 3, time taken = 417.867s

Epoch 4
-—-—-—
Iter 0, train loss = 0.11

In [0]:
# copy model state dict
!cp dense121-dogs-cats.pth '../drive/My Drive/Colab Notebooks/small projects/dogs-cats'